In [213]:
!pip install transformers

In [214]:
pip install py_vncorenlp

Note: you may need to restart the kernel to use updated packages.


In [215]:
from transformers import logging
logging.set_verbosity_error()

In [216]:
import pandas as pd
data = pd.read_csv("/kaggle/input/train-format/trainformat.csv")
print(data['Comment'][0:10])

0    Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...
1    Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...
2    Thời tiết lạnh như này, cả nhà rủ nhau đến leg...
3    Em có đọc review thấy mng bảo trà sữa nướng đề...
4    Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...
5    Đồ ăn ngon, đặt qua now shop để k cẩn thận bị ...
6    Nhà hàng yêu thích của mình đây nè! Đến đây mì...
7    🔸Vị trí: mặt đường Trung Yên rất dễ tìm, một d...
8    Mình đã ăn tại đây và rất ngon,\r\nChuẩn vị ph...
9    Thịt mềm, sốt ngon, dưa góp ăn cùng đỡ ngán\r\...
Name: Comment, dtype: object


In [217]:
crawled_data = pd.read_csv("/kaggle/input/test-crawl/crawler_1567_comments.csv")
print(crawled_data['Comment'][0:10])

0    Mình vừa thử trưa nay. Điểm cộng đầu tiên là b...
1    Trà táo 35k\nCookie socola 38k \nNước ở đây bì...
2    Quá trưa định đi thử Phở Thìn ở Lò Đúc cơ, như...
3    Bình thường ko hay ăn phở Lý quốc sư đâu nhưng...
4    Có cô quen cho văn phòng mình voucher giảm giá...
5    Quá ngon và rẻ cho một cuộc tình chỉ 45k chỗ n...
6    Trước ở gần nên ăn ở đây suốt. Ngay đối diện t...
7    Trước đi qua khu BK thì thấy quán nên bọn mình...
8    Nói chung là ngon nhưng mấy bạn đi ăn lần đầu ...
9    Trời oi oi thèm uống Smoothie thì rẽ qua Nguyễ...
Name: Comment, dtype: object


In [218]:
import torch
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

In [219]:
len(tokenizer.get_vocab())

64001

In [220]:
corpus = []
for tmp in zip(data["Comment"], data["Rating"]):
  sent, lb = tmp
  label = 1
  if lb < 6.0:
    label = 0
  corpus.append((str(sent), label))
#   if(float(lb) == 1):
#     corpus.append((sent, 1))
#   if float(lb) == 0:
#     corpus.append((sent, 0))
print(corpus[0])
print(len(corpus))

# for tmp in zip(crawled_data["Comment"], crawled_data["Rating"]):
#   sent, lb = tmp
#   label = 1
#   if lb < 6.0:
#     label = 0
#   corpus.append((str(sent), label))
# #   if(float(lb) == 1):
# #     corpus.append((sent, 1))
# #   if float(lb) == 0:
# #     corpus.append((sent, 0))
# print(corpus[0])
# print(len(corpus))


('Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trông rất thích', 1)
9071


In [221]:
def segment(corpus):
  ct = []
  lb = []
  for content, label in corpus:
#     if(type(content) == str):
      ct.append(content)
      lb.append(label)
  return ct, lb

In [222]:
sents, labels = segment(corpus)
print(sents[0], " ", labels[0])

Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trông rất thích   1


In [224]:
print(len(sents))

9071


In [225]:
import numpy as np

In [226]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(86)

In [227]:
k = 0
for sent in sents:
  if(sent.find("bình luận không xác định") != -1):
    print(sent)
    k+=1
k

0

In [228]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
def check_spell(string):
  for c in string :
    if c in s1:
      
      return True
  return False

checked_sents = []

for sent in sents:
  if(not check_spell(sent)):
    checked_sents.append(sent)

print(len(checked_sents))

86


In [229]:
for i in range(10):
  print(i, checked_sents[i])

0 1st time came here to try and really satisfy with the foods. Fast and nice services from staffs. 
I also love the design of the restaurant, make it so comfortable and relax for people to have meal here.
1 I personally love indian food and Tandoor Resteraunt offers some really great indian dishes. I decided to go with Gobi Pakora for my starter which came with a mint chutney and spicy veggie chutney (I wasn't too sure what was in it) It's basically cauliflour fritters and it was quite good I can honestly say.
For my main I decided on Chicken Biryani which is flavored rice with spiced chicken. Awesome.

However the lighting on the second floor was not too good but it was alright.
2 Abc
3 Nice place to enjoy and relax. Foods are very delicious and fresh, especially salats. I'm very impressed in cocktails. All staffs are so great and always smile. In here, i have a feeling that i am travelling in Paris. If i have chance to come back Hanoi, i sure come back Ciel ❤❤❤❤❤❤❤❤❤❤❤❤❤
4 Very decen

In [231]:
from sklearn.model_selection import train_test_split

train_sents, valid_sents, train_labels, valid_labels = train_test_split(sents, labels, test_size=0.2)
print(len(train_sents), " ", len(valid_sents))

7256   1815


In [232]:
tmp_sents = []
tmp_labels = []
for tmp in zip(crawled_data["Comment"], crawled_data["Rating"]):
  sent, lb = tmp
  label = 1
  if lb < 6.0:
    label = 0
  
  tmp_sents.append(str(sent))
  tmp_labels.append(label)
#   if(float(lb) == 1):
#     corpus.append((sent, 1))
#   if float(lb) == 0:
#     corpus.append((sent, 0))
print(tmp_sents[0])
print(len(tmp_sents))


Mình vừa thử trưa nay. Điểm cộng đầu tiên là bát bún đầy đặn, rất nhiều cá rán, chả cá, chả tôm. Bề bề cũng to và chắc thịt hơn các quán khác. Giá cả rất hợp lý. 40k/bát bún hải sản. Tuy nhiên nước dùng chưa đậm đà lắm. Nhưng có thể coi là tạm ổn, và gần cơ quan mình. Sẽ quay lại
1567


In [233]:
train_sents = train_sents + tmp_sents
train_labels = train_labels + tmp_labels

In [235]:
len(train_sents)

8823

In [236]:
sum_1 = 0
sum_2 = 0
for label in train_labels:
  if label == 0:
    sum_1 += 1
  else:
    sum_2 += 1
print(sum_1, " ", sum_2)
sum_1 = 0
sum_2 = 0
for label in valid_labels:
  if label == 0:
    sum_1 += 1
  else:
    sum_2 += 1
print(sum_1, sum_2)

1890   6933
379 1436


In [237]:
print(train_sents[0])
print( train_labels[0])

Hơi buồn một tí là tớ biết quán này lại do một anh bạn người nước ngoài giới thiệu, còn mình là người bàn xứ lại không khám phá ra :( Nhân viên ở đây vô cùng thân thiện, bắn tiếng Anh cứ phải gọi là tằng tằng luôn ấy, giới thiệu các món chuyên nghiệp thật. Mỗi tội cái là quán đông khách nên phục vụ hơi lâu tí, với vả thỉnh thoảng đến bị hết chỗ :( Foody xếp quán này là cafe nhưng thực chất là nhà hàng, mình thấy khách đến đây ăn nhiều hơn là uống. Họ phục vụ cả món Việt lẫn món Âu nhưng món Việt có vẻ ngon hơn. Bún nem ở đây được anh bạn tớ khen lắm, tại nhìn cái nem rất đẹp, vàng đều chứ không bị lem như ở nhà mình. Nhân nem cũng đậm đà, nhuyễn thịt. Nước chấm người ta pha cũng khéo nữa. Nói chung là với các bác tây không thích ăn vỉa hè thì đến đây là chuẩn rồi.
1


In [239]:
import re
from keras_preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from gensim.utils import simple_preprocess

In [240]:
pip install underthesea

Note: you may need to restart the kernel to use updated packages.


In [241]:
sw = []
with open("/kaggle/input/vn-stopword/vn_stopword.txt", encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    sw.append(line.replace("\n",""))

In [242]:
from underthesea import text_normalize

In [243]:
text_normalize("quân   khôngđược   thông minh lắm!")

'quân khôngđược thông minh lắm !'

In [244]:
def purify(text):
  vietnamese_characters = "a-zA-Z0-9ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂẾưăạảấầẩẫậắằẳẵặẹẻẽềềểếỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳýỵỷỹ"
  text = text.lower()
  text = text.strip()
  # text = " "+ text + " "
  text = re.sub("[+]", ' ', text) #all math-operating characters removed
  text = re.sub(" \\[a-z]", ' ', text)# all newline and slash characters removed
  
  text = re.sub('[^' + vietnamese_characters + ']+', ' ', text) # all special characters removed
  text = re.sub("([A-Za-z]+[0-9]+)|([0-9]+[A-Za-z]+)", " ", text) #150g
  text = re.sub("[0-9]{3,}", " ", text) #all too long numbers (>= 3 digits) removed
  text = re.sub(r'(\D)\1{2,}', r'\1', text) # all consecutive duplicated characters removed
  text = text.strip()

  #all single-charater words removed
  tmp_text = []
  for x in text.split(" "):
    if(len(x) > 1):
      tmp_text.append(x)
  
  text = " ".join(tmp_text)
  text = text_normalize(text)
  return text

# abc = "Mình order qua Now với giá 55k (xôi + trà quất 10k, mình chỉ áp đc code freeship) mà nhìn còn tệ hơn hộp xôi 20k bình thường! Không biết quán tốn tiền thuê mặt bằng hay sao mà lại cho khách có tẹo xôi + ít ruốc tép + gà xào nấm mà thấy được mấy thịt mấy nấm không? Vị ổn, hộp sạch sẽ, giao hàng rất nhanh nhưng nhận 1 hộp xôi đắt x2 mà chỉ được từng này thì quá thất vọng, chưa kể trà quất uống vị như có xà phòng rất ghê"
# abc = " chả bao giờ quan tâm mấy cái kiểu bình luận như này nhưng phải chấm bút thái độ 10 đ đồ ăn 9 đ nói chung là khá ổn"
abc = "前回も頼んだチキンのグリルと茄子のグリルも安定の美味しさ！茄子の味付けが以前より少し甘味が強く感じました、チキンに添えてあるココナッツ風味の揚げたも大好き！"
abc = "Ngoooooooooooooonnnnnnnnnnnnnnnnnnnnnnnnn nnnnnnnnnnnnnnnnnnnhhhhhhhhhhhhhhhhhheeeeeeeeeeeeeeeee"
# abc = "a b c"
abc = purify(abc)
print(abc)

ngon nhe


In [245]:
#@title
from underthesea import word_tokenize

In [246]:
def preprocess(purified_sents, max_len=None):
  preprocessed_sents = []
  for purified_text in purified_sents:

    #remove all 0-long sents
    # words = text.split(" ")
    # if(len(words) <= 0):
    #   continue

#     preprocessed by word_tokenize of underthesea
    text = word_tokenize(purified_text, format = "text")
    tmp_text = text
    
#     replace stopword by <pad>
#     words = tmp_text.split(" ")
#     for i, word in enumerate(words):
#         if word in sw:
#             words[i] = '<pad>'
#     tmp_text = " ".join(words)

    #preprocessed by rdrsegmenter of vncorenlp and bpe
    # text = rdrsegmenter.word_segment(purified_text)
    # text = bpe.encode(text)
    # for x in text:
    #   print(x)
    # tmp_text =' '.join([' '.join(x) for x in text])

    # preprocessed by py_rdrsegmenter of py_vncorenlp
#     text = py_rdrsegmenter.word_segment(purified_text)
#     tmp_text = " ".join(text)

    if(max_len is not None):
      words = tmp_text.split(" ")
      if(len(words) > max_len):
        words = words[0:max_len]
      tmp_text = " ".join(words)
      
    preprocessed_sents.append(tmp_text)
  return preprocessed_sents


In [247]:
def encode_plus(preprocessed_sents,max_len=256):
  encoded_sents = []
  masks = []
  for sent in preprocessed_sents:
    sent_info = tokenizer.encode_plus(sent,
                                      padding='max_length',
                                      truncation=True,
                                      add_special_tokens=True,
                                      max_length = max_len,
                                      return_token_type_ids=False,
                                      return_attention_mask=True,
                                     return_tensors='np')
    encoded_sent = sent_info['input_ids'].flatten()
    mask =  sent_info['attention_mask'].flatten()
    encoded_sents.append(encoded_sent)
    masks.append(mask)
  return encoded_sents, masks

en

In [248]:
train_purified_sents = [purify(train_sent) for train_sent in train_sents]
valid_purified_sents = [purify(valid_sent) for valid_sent in valid_sents]

In [34]:
# for i, sent in enumerate(train_purified_sents):
#   if(sent == ""):
#     train_purified_sents[i] = "bình luận không xác định"

In [249]:
# for i, sent in enumerate(valid_purified_sents):
#   if(sent == ""):
#     valid_purified_sents[i] = "bình luận không xác định"

IndentationError: unexpected indent (1899538612.py, line 2)

In [249]:
train_preprocessed_sents = preprocess(train_purified_sents)
valid_preprocessed_sents = preprocess(valid_purified_sents)

In [250]:
print(valid_preprocessed_sents[535])

khá ổn nhưng thiếu nước_canh phở ăn khô mồm quá_trời nắng thế này nhai cơm khô sao nổi góp lần sau đừng làm mình that vong


In [251]:
train_ids, train_masks = encode_plus(train_preprocessed_sents, max_len=256)
valid_ids, valid_masks = encode_plus(valid_preprocessed_sents, max_len=256)

In [252]:
import numpy as np

In [255]:
batch_size = 32
train_inputs = torch.tensor(train_ids)
valid_inputs = torch.tensor(valid_ids)
train_labels = torch.tensor(train_labels)
valid_labels = torch.tensor(valid_labels)
train_masks = torch.tensor(train_masks)
valid_masks = torch.tensor(valid_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = RandomSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sou

In [259]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [260]:
device

device(type='cuda')

In [261]:
from matplotlib import pyplot as plt

In [262]:
print(count)

48


In [263]:
import torch.nn as nn
import torch.nn.functional as F

In [264]:
class textCNN(nn.Module):
  def __init__(self,seq_len=256, embedding_size=768, output_size=2):
    super(textCNN, self).__init__()
    self.embedding_size = embedding_size
    self.output_size = output_size
    self.seq_len = seq_len

    kernel_1 = (2, embedding_size)
    kernel_2 = (3, embedding_size)
    kernel_3 = (4, embedding_size)
    kernel_4 = (5, embedding_size)
    self.convo_1 = nn.Conv2d(1, 256, kernel_1)
    self.convo_2 = nn.Conv2d(1, 256, kernel_2)
    self.convo_3 = nn.Conv2d(1, 256, kernel_3)
    self.convo_4 = nn.Conv2d(1, 256, kernel_4)
    self.fc1 = nn.Linear(4*self.seq_len, 2, bias=True)

  def forward(self, inputs):

    out_1 = self.convo_1(inputs)
    out_2 = self.convo_2(inputs)
    out_3 = self.convo_3(inputs)
    out_4 = self.convo_4(inputs)

    out_1 = nn.functional.max_pool2d(out_1,(self.seq_len - 1, 1))
    out_2 = nn.functional.max_pool2d(out_2,(self.seq_len - 2, 1))
    out_3 = nn.functional.max_pool2d(out_3,(self.seq_len - 3, 1))
    out_4 = nn.functional.max_pool2d(out_4,(self.seq_len - 4, 1))

    out_1 = torch.squeeze(out_1)
    out_2 = torch.squeeze(out_2)
    out_3 = torch.squeeze(out_3)
    out_4 = torch.squeeze(out_4)
    
    # print(out_1.shape)
    # print(out_2.shape)
    # print(out_3.shape)
    # print(out_4.shape)
    
    out = torch.cat((out_1, out_2, out_3, out_4), -1)
    
    out = self.fc1(out)
    out = torch.sigmoid(out)
    return out


In [265]:
class simple_textCNN(nn.Module):
  def __init__(self, seq_len=256, embedding_size=768,kernel_size=3, output_size=2):
    super(simple_textCNN, self).__init__()

    self.embedding_size = embedding_size
    self.output_size = output_size
    self.seq_len = seq_len
    self.kernel_size = kernel_size
    kernel = (self.kernel_size, self.embedding_size)
    
    self.convo = nn.Conv2d(1, 256, kernel)

    self.fc1 = nn.Linear(self.seq_len, 2, bias=True)
    
    nn.init.normal_(self.fc1.weight, std=0.02)
    nn.init.normal_(self.fc1.bias, 0)
    nn.init.normal_(self.convo.weight, std=0.02)
    
    
  def forward(self, inputs):

    out = self.convo(inputs)
    # out = torch.t(out)
#     print(out.shape)
    out = F.max_pool2d(out,(self.seq_len - self.kernel_size + 1, 1))
#     print(out.shape)
    out = torch.squeeze(out)
#     print(out.shape)
    out = self.fc1(out)
    out = torch.sigmoid(out)

    return out

In [266]:
class BERT_for_sentiment(nn.Module):
  def __init__(self, output_size=2, dropout=0.1):
    super(BERT_for_sentiment, self).__init__()
    self.output_size = output_size
    self.dropout = dropout
    self.bert = AutoModel.from_pretrained("vinai/phobert-base")
    self.fc1 = nn.Linear(768, 768, bias=True)
    self.layernorm_1 = nn.LayerNorm((768,), eps = 1e-05, elementwise_affine=True)
    self.fc2 = nn.Linear(768, self.output_size, bias=True)
    nn.init.normal_(self.fc1.weight, std=0.02)
    nn.init.normal_(self.fc1.bias, 0)
    nn.init.normal_(self.fc2.weight, std=0.02)
    nn.init.normal_(self.fc2.bias, 0)
    
  def forward(self, inputs, attention_mask):
    x = self.bert(input_ids=inputs, attention_mask=attention_mask)
    x = x.pooler_output
#     x = F.dropout(x, self.dropout, inplace=False)
#     x = self.fc1(x)
#     x = self.layernorm_1(x)
#     x = F.dropout(x, self.dropout, inplace=False)
    x = self.fc2(x)
    out = F.softmax(x, dim=-1)
    
    return out
    

In [267]:
class BERT_for_sentiment_large(nn.Module):
  def __init__(self, output_size=2, dropout=0.1):
    super(BERT_for_sentiment_large, self).__init__()
    self.output_size = output_size
    self.dropout = dropout
    self.bert =  AutoModel.from_pretrained("vinai/phobert-base", num_labels=2, output_hidden_states=True)
    self.fc1 = nn.Linear(768, 768, bias=True)
    self.layernorm_1 = nn.LayerNorm((768,), eps = 1e-05, elementwise_affine=True)
    self.fc2 = nn.Linear(4*768, self.output_size, bias=True)
    nn.init.normal_(self.fc1.weight, std=0.02)
    nn.init.normal_(self.fc1.bias, 0)
    nn.init.normal_(self.fc2.weight, std=0.02)
    nn.init.normal_(self.fc2.bias, 0)
    
  def forward(self, inputs, attention_mask):
    outputs = self.bert(input_ids=inputs, attention_mask=attention_mask)
    x = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
#     x = F.dropout(x, self.dropout, inplace=False)
#     x = self.fc1(x)
#     x = self.layernorm_1(x)
#     x = F.dropout(x, self.dropout, inplace=False)
    x = self.fc2(x)
#     out = F.softmax(x, dim=-1)
    out = x
    return out
    

In [268]:
import torch

In [269]:
class BERT_plus_LSTM_sentiment(nn.Module):
  def __init__(self, hidden_size,n_layers=1, output_size=2, bidirectional=True, dropout=0.1):
    super(BERT_plus_LSTM_sentiment, self).__init__()
    self.output_size = output_size
    self.dropout = dropout
    self.bert = AutoModel.from_pretrained("vinai/phobert-base")
    self.bidirectional = bidirectional
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    self.LSTM = nn.LSTM(768, self.hidden_size, num_layers=self.n_layers, bidirectional=self.bidirectional, batch_first=True)
    if self.bidirectional == True:
      self.hidden_size = hidden_size*2
    
    self.fc1 = nn.Linear(self.hidden_size, 64, bias=True)
    self.fc2 = nn.Linear(64, self.output_size, bias=True)
    
    nn.init.normal_(self.fc1.weight, std=0.02)
    nn.init.normal_(self.fc1.bias, 0)
    
    nn.init.normal_(self.fc2.weight, std=0.02)
    nn.init.normal_(self.fc2.bias, 0)
    
  def forward(self, inputs, attention_mask):

    x = self.bert(input_ids=inputs, attention_mask=attention_mask)
    x = x[0]
    
    out_lstm, _ = self.LSTM(x)
    max_pool, _ = torch.max(out_lstm, 1)
    linear_out = F.dropout(torch.relu(self.fc1(max_pool)), self.dropout, inplace=False)
    linear_out = self.fc2(linear_out)
    out = torch.sigmoid(linear_out)
    
    return out

In [270]:
class BERT_plus_textCNN_sentiment(nn.Module):
  def __init__(self, output_size=2):
    super(BERT_plus_textCNN_sentiment, self).__init__()
    self.output_size = output_size
    self.bert = AutoModel.from_pretrained("vinai/phobert-base")

    self.CNN = simple_textCNN(kernel_size=3, output_size=self.output_size)

  def forward(self, inputs, attention_mask):
    x = self.bert(input_ids=inputs, attention_mask=attention_mask)
    x = x.last_hidden_state
    
    x = torch.squeeze(x)
    x = torch.unsqueeze(x, dim=1)
    out= self.CNN(x)
    return out

In [271]:

model = BERT_for_sentiment_large(output_size=2)
# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)
# model = BERT_plus_LSTM_sentiment(64, n_layers=1, output_size=2)
# model = BERT_plus_textCNN_sentiment(output_size=2)
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)





In [272]:
for i, child in enumerate(model.children()):
    print("i = ", i, " ", child)

i =  0   RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropo

In [103]:
# for i, child in enumerate(model.bert.embeddings.children()):
#     print("i = ", i, " ", child)
#     for params in child.parameters():
#         params.requires_grad = False

In [104]:
# @title
# # print(model.children())
# tmp = 0
# for i, child in enumerate(model.bert.encoder.layer.children()):
#     if i < 8:
#         for params in child.parameters():
#             params.requires_grad = False
# #     print("i =", i," ", child)
#     tmp += 1
# print(tmp)

In [273]:
count = 0
for params in model.parameters():
  count+=1
print(count)

205


In [274]:
count = 0
for params in model.parameters():
  if(params.requires_grad == True):
    # print(params)
    count+=1

print(count)

205


In [275]:
import torch.nn as nn
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup

max_epochs = 10
learning_rate = 2*1e-5

weight_decay = 0.01
optimizing_parameters = filter(lambda p: p.requires_grad, model.parameters())
criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(
    optimizing_parameters,
    lr=learning_rate,
    weight_decay=weight_decay
)
lr_scheduler = get_linear_schedule_with_warmup(
                optimizer, 
                num_warmup_steps=0, 
                num_training_steps=len(train_dataloader)*max_epochs
            )

In [278]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [279]:
device

device(type='cuda')

In [280]:
model = model.to(device)

In [281]:
from sklearn.metrics import roc_auc_score

def uac(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return roc_auc_score(pred_flat, labels_flat)

In [282]:
from tqdm import tqdm, tqdm_notebook

In [283]:
def train_epoch(model,criterion, optimizer,scheduler, train_loader):
  total_loss = 0
  total_acc = 0
  total = 0
  tmp = 0
  model.train()
  
  for batch in tqdm(train_loader):
    input_ids, input_mask, labels = batch

    input_ids = input_ids.to(device)    
    input_mask = input_mask.to(device)   
    labels = labels.to(device)

    optimizer.zero_grad()
    output = model(input_ids, attention_mask=input_mask)
#     output = output.logits
    loss = criterion(output, labels)

    # loss = output[0]
    # logits = output[1]

    loss.backward()
    
    nn.utils.clip_grad_norm_(optimizing_parameters, max_norm=1.0)
    optimizer.step()
    scheduler.step()
    
    
    total_loss += loss.item()
    total += len(labels)

    logits = output.detach().cpu().numpy()
    labels = labels.to('cpu').numpy()
    acc = flat_accuracy(logits, labels)

    total_acc += acc

    tmp += 1

  return total_loss/total, total_acc / tmp 

In [284]:
def validate_epoch(model,criterion,valid_loader):
    model.eval()
    total_loss = total = 0
    accuracy = 0
    eval_f1 = 0
    tmp = 0
    for batch in tqdm(valid_loader):
      # batch = torch.toTensor(t.to(device) for t in batch)
      input_ids, input_mask, labels = batch

      input_ids = input_ids.to(device=device)
      input_mask = input_mask.to(device=device)
      labels = labels.to(device=device)

          # Forwards pass
      with torch.no_grad():
        logits = model(input_ids, attention_mask=input_mask)
#         logits = logits.logits
          
          # Calculate how wrong the model is
        # loss = logits[0]
        loss = criterion(logits, labels)
        
        # print(loss)
        logits = logits.detach().cpu().numpy()
        labels = labels.to('cpu').numpy()

          
        acc = flat_accuracy(logits, labels)
        accuracy += acc
        # Record metrics
        total_loss += loss.item()
        total += len(labels)
        tmp += 1
    return total_loss / total , accuracy / tmp

In [285]:
max_epochs = 10
n_epochs = 0
train_losses, valid_losses = [], []
train_acces, valid_acces = [], []
for _ in range(max_epochs):
    train_loss, train_acc = train_epoch(model,criterion, optimizer,lr_scheduler, train_dataloader)
    valid_loss, valid_acc = validate_epoch(model,criterion, valid_dataloader)
    
    tqdm.write(
        f'epoch #{n_epochs + 1:3d}\n'
        f'train_loss: {train_loss:.2e}'
        f'\ttrain_acc: {train_acc:.2e}\n'
        f'valid_loss: {valid_loss:.2e}'
        f'\tvalid_acc: {valid_acc:.2e}\n',
    )
    # print("epoch:", " ", n_epochs)
    # print("train_loss: ", " ", train_loss)
    # print("valid_loss: ", " ", valid_loss)
    # print("valid_acc: ", " ", valid_acc)
    # Early stopping if the current valid_loss is greater than the last three valid losses
#     if len(valid_losses) > 2 and all(valid_loss >= loss
#                                      for loss in valid_losses[-3:]):
#         print('Stopping early')
#         break
#     if len(valid_losses) == 0:
#       print("saving 1st model:...")
#       torch.save(model.state_dict(), '/kaggle/working/phobert_linear_x4.pt')
#     else:
#       min_n = 10
#       max_acc = -1
#       for loss in valid_losses:
#         if(min_n > loss):
#          min_n = loss
        
#       if valid_loss < min_n:
#         print("saving the best model for best loss:...")
#         torch.save(model.state_dict(), '/kaggle/working/phobert_linear_x4.pt')
#       elif valid_loss == min_n and abs(train_acc - valid_acc) < 0.02:
#         print("saving the best model for best fit:...")
#         torch.save(model.state_dict(), '/kaggle/working/phobert_linear_x4.pt')
    print("saving model:...")
    torch.save(model.state_dict(), f'/kaggle/working/phobert_linear_train_crawl_{n_epochs+1}.pt')
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_acces.append(train_acc)
    valid_acces.append(valid_acc)
    n_epochs += 1

100%|██████████| 57/57 [00:13<00:00,  4.22it/s]


epoch #  1
train_loss: 8.63e-03	train_acc: 8.94e-01
valid_loss: 6.72e-03	valid_acc: 9.27e-01

saving model:...


100%|██████████| 57/57 [00:13<00:00,  4.22it/s]


epoch #  2
train_loss: 6.27e-03	train_acc: 9.34e-01
valid_loss: 7.29e-03	valid_acc: 9.15e-01

saving model:...


100%|██████████| 57/57 [00:13<00:00,  4.22it/s]


...

epoch #  7
train_loss: 2.46e-03	train_acc: 9.79e-01
valid_loss: 8.41e-03	valid_acc: 9.22e-01

saving model:...


100%|██████████| 57/57 [00:13<00:00,  4.22it/s]


epoch #  8
train_loss: 2.02e-03	train_acc: 9.83e-01
valid_loss: 8.73e-03	valid_acc: 9.23e-01

saving model:...


 12%|█▏        | 32/276 [00:24<03:05,  1.31it/s]


KeyboardInterrupt: 

In [286]:
gc.collect()

6024

In [287]:
from tqdm import tqdm

In [288]:
test_model = BERT_for_sentiment_large()

In [289]:
test_model.load_state_dict(torch.load("/kaggle/working/phobert_linear_train_crawl_3.pt"))

<All keys matched successfully>

In [134]:
phobert = AutoModel.from_pretrained("vinai/phobert-base")

In [ ]:
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.29it/s]
epoch #  1
train_loss: 1.38e-02	train_acc: 8.74e-01
valid_loss: 1.27e-02	valid_acc: 9.16e-01

saving 1st model:...
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.29it/s]
epoch #  2
train_loss: 1.25e-02	train_acc: 9.13e-01
valid_loss: 1.25e-02	valid_acc: 9.17e-01

saving the best model for best loss:...
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.29it/s]
epoch #  3
train_loss: 1.20e-02	train_acc: 9.31e-01
valid_loss: 1.24e-02	valid_acc: 9.23e-01

saving the best model for best loss:...
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.28it/s]
epoch #  4
train_loss: 1.20e-02	train_acc: 9.32e-01
valid_loss: 1.24e-02	valid_acc: 9.24e-01

saving the best model for best loss:...
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.29it/s]
epoch #  5
train_loss: 1.17e-02	train_acc: 9.39e-01
valid_loss: 1.26e-02	valid_acc: 9.16e-01

100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.29it/s]
epoch #  6
train_loss: 1.17e-02	train_acc: 9.41e-01
valid_loss: 1.25e-02	valid_acc: 9.19e-01

100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.28it/s]
epoch #  7
train_loss: 1.14e-02	train_acc: 9.48e-01
valid_loss: 1.23e-02	valid_acc: 9.27e-01

saving the best model for best loss:...
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.29it/s]
epoch #  8
train_loss: 1.14e-02	train_acc: 9.51e-01
valid_loss: 1.22e-02	valid_acc: 9.29e-01

saving the best model for best loss:...
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.28it/s]
epoch #  9
train_loss: 1.13e-02	train_acc: 9.53e-01
valid_loss: 1.22e-02	valid_acc: 9.33e-01

saving the best model for best loss:...
100%|██████████| 256/256 [03:08<00:00,  1.36it/s]
100%|██████████| 29/29 [00:06<00:00,  4.29it/s]
epoch # 10
train_loss: 1.13e-02	train_acc: 9.54e-01
valid_loss: 1.22e-02	valid_acc: 9.30e-01


In [290]:
torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1).cpu().detach().numpy()

NameError: name 'outputs' is not defined

In [291]:
trained_phobert = test_model.bert
# trained_phobert = AutoModel.from_pretrained("vinai/phobert-base", num_labels=2, output_hidden_state=True)
trained_phobert.to(device)
trained_phobert.eval()
total_loss = total = 0
accuracy = 0
eval_f1 = 0
tmp = 0
X_train = []
y_train = []
X_test = []
y_test = []
for batch in tqdm(train_dataloader):
    # batch = torch.toTensor(t.to(device) for t in batch)
    input_ids, input_mask, labels = batch

    input_ids = input_ids.to(device=device)
    input_mask = input_mask.to(device=device)
    labels = labels.to(device=device)

          # Forwards pass
    with torch.no_grad():
      logits = trained_phobert(input_ids, attention_mask=input_mask)
      logits = torch.cat((logits[2][-1][:,0, ...],logits[2][-2][:,0, ...], logits[2][-3][:,0, ...], logits[2][-4][:,0, ...]),-1)
#       logits = logits.pooler_output
        
      logits = logits.detach().cpu().numpy()
      labels = labels.to('cpu').numpy()

      for logit in logits:
        X_train.append(logit)
      for label in labels:
        y_train.append(label)

for batch in tqdm(valid_dataloader):
    # batch = torch.toTensor(t.to(device) for t in batch)
    input_ids, input_mask, labels = batch

    input_ids = input_ids.to(device=device)
    input_mask = input_mask.to(device=device)
    labels = labels.to(device=device)

          # Forwards pass
    with torch.no_grad():
      logits = trained_phobert(input_ids, attention_mask=input_mask)
      logits = torch.cat((logits[2][-1][:,0, ...],logits[2][-2][:,0, ...], logits[2][-3][:,0, ...], logits[2][-4][:,0, ...]),-1)
#       logits = logits.pooler_output
          
        # Calculate how wrong the model is
        # loss = logits[0]
        # print(loss)
      logits = logits.detach().cpu().numpy()
      labels = labels.to('cpu').numpy()

      for logit in logits:
        X_test.append(logit)
      for label in labels:
        y_test.append(label)

100%|██████████| 57/57 [00:13<00:00,  4.21it/s]


In [83]:
# X = X_train + X_test
# y = y_train + y_test

In [296]:
from sklearn.decomposition import PCA
pca = PCA(n_components=900)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [297]:
pca.explained_variance_ratio_.sum()

0.9968728644297893

In [298]:
from sklearn.metrics import accuracy_score

In [299]:
from sklearn.tree import DecisionTreeClassifier

In [118]:
# classifier = DecisionTreeClassifier()
# classifier.fit(np.array(X_train), np.array(y_train))
# print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(classifier.predict(X_train), y_train)))
# print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(classifier.predict(X_test), y_test)))

Vanilla Implementation : Accuracy score for training data : 0.9995099840744824
Vanilla Implementation : Accuracy score for testing data : 0.7761852260198456


In [300]:
from sklearn.model_selection import cross_val_score

In [301]:
from sklearn.ensemble import RandomForestClassifier
# RFclassifier = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
# RFclassifier.fit(X_train, y_train)
# print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(RFclassifier.predict(X_train), y_train)))
# print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(RFclassifier.predict(X_test), y_test)))

In [302]:
from sklearn.ensemble import GradientBoostingClassifier
# NUM_CLASSIFIERS = 40
# MAX_DEPTH = 4
# GBCclassifier = GradientBoostingClassifier()
# GBCclassifier.fit(X_train, y_train)
# print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(GBCclassifier.predict(X_train), y_train)))
# print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(GBCclassifier.predict(X_test), y_test)))
# cross_val_score(GBCclassifier, X, y).mean()
# print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(classifier.predict(X_train), y_train)))
# print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(classifier.predict(X_test), y_test)))

In [303]:
from xgboost import XGBClassifier
# XGBclassifier = XGBClassifier()
# XGBclassifier.fit(X_train, y_train)
# print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(XGBclassifier.predict(X_train), y_train)))
# print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(XGBclassifier.predict(X_test), y_test)))
# cross_val_score(XGBclassifier, X, y).mean()

In [304]:
from lightgbm import LGBMClassifier
# LGBMclassifier = LGBMClassifier()
# # cross_val_score(LGBMclassifier, X, y).mean()
# LGBMclassifier.fit(X_train, y_train)
# print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(LGBMclassifier.predict(X_train), y_train)))
# print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(LGBMclassifier.predict(X_test), y_test)))

In [305]:
device

device(type='cuda')

In [99]:
from sklearn.ensemble import VotingClassifier
tmp_vote = VotingClassifier([('clf1', RandomForestClassifier()),
                        ('clf2', XGBClassifier()),
                        ('clf3', LGBMClassifier())], voting="hard")
tmp_vote.fit(X_train_pca, y_train)
print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(tmp_vote.predict(X_train_pca), y_train)))
print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(tmp_vote.predict(X_test_pca), y_test)))

Vanilla Implementation : Accuracy score for training data : 0.9995099840744824
Vanilla Implementation : Accuracy score for testing data : 0.9228224917309813


In [158]:
from sklearn.linear_model import LogisticRegression

In [306]:
from sklearn.ensemble import VotingClassifier
dcm_vote = VotingClassifier([('clf1', RandomForestClassifier()),
                        ('clf2', XGBClassifier()),
                        ('clf3', LGBMClassifier()),
                        ('clf4', LogisticRegression())], voting="soft")
dcm_vote.fit(X_train_pca, y_train)
print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(dcm_vote.predict(X_train_pca), y_train)))
print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(dcm_vote.predict(X_test_pca), y_test)))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Vanilla Implementation : Accuracy score for training data : 0.999659979598776
Vanilla Implementation : Accuracy score for testing data : 0.9234159779614325


In [148]:
from sklearn.ensemble import VotingClassifier
dcm_vote = VotingClassifier([('clf1', RandomForestClassifier()),
                        ('clf2', XGBClassifier()),
                        ('clf3', LGBMClassifier())], voting="soft")
dcm_vote.fit(X_train, y_train)
print("Vanilla Implementation : Accuracy score for training data : {}".format(accuracy_score(dcm_vote.predict(X_train), y_train)))
print("Vanilla Implementation : Accuracy score for testing data : {}".format(accuracy_score(dcm_vote.predict(X_test), y_test)))

In [307]:
test_data = pd.read_csv("/kaggle/input/data-sentiment/test.csv")
print(test_data["RevId"][0:10])

0     781115
1    1219481
2    1703765
3    4870346
4    2638711
5    2288606
6    2369977
7    1395037
8    1388406
9    2704939
Name: RevId, dtype: int64


In [308]:
test_corpus = []
for id, sent in zip(test_data["RevId"], test_data["Comment"]):
  if(type(sent) != str):
    sent = "bình luận không xác định"
  test_corpus.append((id, sent))

print(len(test_corpus))

5103


In [309]:
revIDs = [id for id,_ in test_corpus]
sents = [sent for _,sent in test_corpus]
print(revIDs[0:10])
# print(sents[0])

[781115, 1219481, 1703765, 4870346, 2638711, 2288606, 2369977, 1395037, 1388406, 2704939]


In [310]:
for sent in test_data["Comment"]:
  if(type(sent) != str):
    print(sent)

nan
nan
nan


In [311]:
checked_sents = []
count = 0
for i, sent in enumerate(sents):
  # print(i)
  if(not check_spell(sent)):
    print(i, sent)
    count += 1

print(count)
# for sent in checked_sents:
#   print(sent)

39 Quite beautiful place to stay and near everything we need as a customer/tourist. Next to BIDV TOWER and VIETCOMBANK HQ, near the lake, ancient town....
Breakfast is not as I expected, not that quality for hotel :( quite a shame.
94 Ngon
165 I wish I had this roof in my house.
171 Chilled bar with cheap price in the middle of the famous beer street. If they don't have any special event, you'll need to pay only 100.000 for the entrance ticket (included one drink). 
Its space is quite small but usually crowded. I wonder why there are so many Korean guys around whenever I go here. It seems this bar is the favorite place of Korean people who visit Hanoi. Haha.
215 コメ麺を使った焼きそば。
非常に安い。
470 Like
476 Siu ngonnnn
478 I love the bubble tea here.  It's classy and tasty with several well-balanced flavor selections.  I usually get mint but sometimes I will get taro or chocolate.  I was chided the other day by a comment on a review I made at another tea place.  I had complained about being served 

In [312]:
purified_sents = [purify(str(sent)) for sent in sents]

In [313]:
print(purified_sents[2806])

ngonmnmn


In [168]:
# for i, sent in enumerate(purified_sents):
#   if(sent == ""):
#     purified_sents[i] = "bình luận không xác định"
#     print(i)

In [314]:
preprocessed_sents = preprocess(purified_sents)

In [ ]:
# encoded_sents = encode(preprocessed_sents, max_len=150, max_seq=256)

In [315]:
padded_sents, attention_masks = encode_plus(preprocessed_sents, max_len=256)

In [ ]:
# padded_sents = padding(encoded_sents, max_len)

In [ ]:
# attention_masks = attention_mask(padded_sents)

TypeError: 'str' object is not callable

In [316]:
batch_size = 32

inputs = torch.tensor(padded_sents)
ids = torch.tensor(revIDs)
masks = torch.tensor(attention_masks)

test_data = TensorDataset(ids, masks, inputs)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [317]:
inputs.shape

torch.Size([5103, 256])

In [318]:
from tqdm import tqdm

In [319]:
test_model.to(device)

BERT_for_sentiment_large(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [320]:
X = []
IDs = []
for batch in tqdm(test_dataloader):
    # batch = torch.toTensor(t.to(device) for t in batch)
    ids, input_mask, input_ids = batch

    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    ids = ids.to(device=device)

          # Forwards pass
    with torch.no_grad():
      logits = trained_phobert(input_ids, attention_mask=input_mask)
#       logits = logits.pooler_output
      logits = torch.cat((logits[2][-1][:,0, ...],logits[2][-2][:,0, ...], logits[2][-3][:,0, ...], logits[2][-4][:,0, ...]),-1)
#       logits = torch.cat((logits[2][-1][:,0, ...],logits[2][-2][:,0, ...], logits[2][-3][:,0, ...], logits[2][-4][:,0, ...]),-1)
        # Calculate how wrong the model is
        # loss = logits[0]
        # print(loss)
      ids = ids.detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      for id_ in ids:
        IDs.append(id_)
      for logit in logits:
        X.append(logit)

100%|██████████| 160/160 [00:38<00:00,  4.20it/s]


In [321]:
X_pca = pca.transform(X)

In [322]:
preds = dcm_vote.predict(X_pca)

sub = []
for tmp in zip(IDs, preds):
    sub.append(tmp)

In [110]:
sub = []
test_model.eval()
total_loss = total = 0
for batch in tqdm(test_dataloader):
# batch = torch.toTensor(t.to(device) for t in batch)
  ids, input_mask, input_ids = batch

  input_ids = input_ids.to(device)
  input_mask = input_mask.to(device)
  ids = ids.to(device=device)
          # Forwards pass
  with torch.no_grad():
    outputs = test_model(input_ids, attention_mask=input_mask)
    # loss = outputs[0]
#     outputs = outputs.logits
    loss = outputs.detach().cpu().numpy()
    ids = ids.to('cpu').numpy()

    preds_flat = np.argmax(loss, axis=1).flatten()
    for tup in zip(ids, preds_flat):
      sub.append(tup)


100%|██████████| 160/160 [00:38<00:00,  4.15it/s]


In [323]:
print(sub[0:10])

[(781115, 0), (1219481, 0), (1703765, 1), (4870346, 0), (2638711, 1), (2288606, 1), (2369977, 1), (1395037, 1), (1388406, 1), (2704939, 0)]


In [324]:
pred = [pr for _, pr in sub]

In [325]:
ids = [id for id,_ in sub]

In [326]:
print(len(pred))

5103


In [327]:
tmp_ids = np.array(pred)

In [328]:
print(tmp_ids)

[0 0 1 ... 0 1 1]


In [329]:
print(sub[0:10])

[(781115, 0), (1219481, 0), (1703765, 1), (4870346, 0), (2638711, 1), (2288606, 1), (2369977, 1), (1395037, 1), (1388406, 1), (2704939, 0)]


In [330]:
my_submission = pd.DataFrame({'RevId': np.array(ids).reshape(5103), 'Rating': np.array(pred).reshape(5103)})

In [331]:
my_submission.to_csv("/kaggle/working/submission_13_12_5.csv", index=False)